# Imports

In [ ]:
#uncomment if connected to colab instance instead of local instance
# from google.colab import drive
# drive.mount('/content/gdrive')
# !pip install Ipython --upgrade 
# !pip install pyyaml h5py 
import sys 

In [ ]:
%load_ext autoreload
%autoreload 2
sys.path.insert(1, "C:\\Users\\Yousuf Baker\\Desktop\\ESS_proj")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import scipy.io
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date       
import tensorflow as tf
from datetime import date
import time
from ESS_utils_RT import *
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

# NY ZONES

## Prep Work

In [ ]:
# DATA IMPORTS
dir = "C:\\Users\\Yousuf Baker\\Desktop\\ESS_proj\\NY_data\\"
RTP_NYC = np.array(scipy.io.loadmat(dir + "RTP_NYC_2010_2019.mat")['RTP'])
DAP_NYC = np.array(scipy.io.loadmat(dir + "DAP_NYC_2010_2019.mat")['DAP'])
RTP_LONGIL = np.array(scipy.io.loadmat(dir + "RTP_LONGIL_2010_2019.mat")['RTP'])
DAP_LONGIL = np.array(scipy.io.loadmat(dir + "DAP_LONGIL_2010_2019.mat")['DAP'])
RTP_NORTH = np.array(scipy.io.loadmat(dir + "RTP_NORTH_2010_2019.mat")['RTP'])
DAP_NORTH = np.array(scipy.io.loadmat(dir + "DAP_NORTH_2010_2019.mat")['DAP'])
RTP_WEST = np.array(scipy.io.loadmat(dir + "RTP_WEST_2010_2019.mat")['RTP'])
DAP_WEST = np.array(scipy.io.loadmat(dir + "DAP_WEST_2010_2019.mat")['DAP'])
  
RTP = RTP_NYC
DAP = DAP_NYC

### SET PARAMS HERE

In [ ]:
#Generating GT Value Functions
'''
Select dates: The data ends on 2019/12/31. We take the data range 2017/1/1 to 2018/12/31
'''

Ts = 1/12 # time step: 5min

# Last day for New England
lastDay = date.toordinal(date(2019, 12, 31)) + 366 - 1 # 737789


start = date.toordinal(date(2017, 1, 1)) + 366 - 1 # 73669
stop = date.toordinal(date(2018, 12, 31)) + 366 - 1 # 737424
startTest = date.toordinal(date(2019, 1, 1)) + 366 - 1
stopTest = date.toordinal(date(2019, 12, 31)) + 366 - 2
ind1 = (len(RTP_WEST[0])-lastDay+startTest-1)
ind2 = (len(RTP_WEST[0])-lastDay+stopTest+1)
# T = int(((stop-start+1)*24/Ts)) # T: 210240, MATLAB: 210240

# tlambda: real time price over time period t
tlambda = RTP[:, (len(RTP[0])-lastDay+start-2):(len(RTP[0])-lastDay+stop+1)] # (288, 731)
tlambda = tlambda.flatten('F')
# tlambda_DA: day ahead price over time period t
tlambda_DA = DAP[:, (len(DAP[0])-lastDay+start-2):(len(DAP[0])-lastDay+stop+1)] # (288, 731)
print(len(DAP[0])-lastDay+start-2)
print(len(DAP[0])-lastDay+stop+1)
print(tlambda_DA.shape)
tlambda_DA = tlambda_DA.flatten('F') # (210528,)
T_CNN = 104820 - 288

'''
Set parameters
'''
Pr = 1/12 # normalized power rating wrt energy rating (highest power input allowed to flow through particular equipment)
P = Pr*Ts  # actual power rating taking time step size into account, 0.5*1/12 = 0.041666666666666664
eta = .9  # efficiency
c = 10  # marginal discharge cost - degradation
ed = 0.001  # SoC sample granularity
ef = .5  # final SoC target level, use 0 if none (ensure that electric vehicles are sufficiently charged at the end of the period)
Ne = math.floor(1/ed)+1  # number of SOC samples, (1/0.001)+1=1001
e0 = .5  # Beginning SoC level


'''
Downsample settings
'''
num_segment = 1

2556
3288
(288, 732)


In [ ]:
#for later evaluation
rtp_NYC = RTP_NYC[:, ind1:ind2].flatten('F')
rtp_LONGIL = RTP_LONGIL[:, ind1:ind2].flatten('F')
rtp_NORTH = RTP_NORTH[:, ind1:ind2].flatten('F')
rtp_WEST = RTP_WEST[:, ind1:ind2].flatten('F')

### 5 Min Valuation

In [ ]:
#GENERATING GT VALUE FUNCTION FOR TRAINING
vAvg = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, tlambda)[1]

In [ ]:
T = rtp_NYC.shape[0] - 288

In [ ]:
#generating test value functions to calculate benchmark profits 5 min valuation
vAvg_NYC = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, rtp_NYC)[1]
vAvg_LONGIL = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, rtp_LONGIL)[1]
vAvg_NORTH = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, rtp_NORTH)[1]
vAvg_WEST = generate_value_function(Ts, P, eta, c, ed, ef, Ne, T, num_segment, rtp_WEST)[1]

Time: 14.96348237991333
Time: 14.786789417266846
Time: 14.785568237304688
Time: 14.934889316558838


## Training

### RTP + DAP

In [ ]:
#Training + Parameter Control
num_DAP = 24
num_RTP = 36

#setting test dir info
folder = 'C:\\Users\\Yousuf Baker\\Desktop\\ESS_proj\\'

region = 'NY'
test = 'CNN_LSTM_' + str(num_segment) +'_0.5'
net = 'vanilla' #set string as t1 or vanilla based on which model you want


#uncomment below to generate training set 
# X, y = generate_train_CNN(T, DAP, tlambda, 
#                           start, stop, lastDay, 
#                           num_DAP, num_RTP, vAvg)
# X = np.asarray(X)
# y = np.asarray(y)
# x_train, x_val, y_train, y_val = train_test_split(X, y, 
#                                                   test_size=0.05, shuffle=True)

model = val_CNN_LSTM(output_size=num_segment, net=net)
# model.summary()

In [ ]:
curr_run = net + '_best_model_epoch{epoch:02d}_loss{val_loss:.5f}.hdf5'
checkpoint_path = folder + region + '_tests\\' + test + '\\' + curr_run

#uncomment for early stopping callback
# es_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     min_delta=0.001,
#     patience=10,
#     verbose=0,
#     mode='auto',
#     baseline=None,
#     restore_best_weights=False
# )


epochs = 100
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_loss', verbose=1, 
                                                 save_best_only=True, mode='min')

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs,batch_size=32, 
                    verbose=1, shuffle =True,
                    callbacks=[cp_callback])

In [ ]:
#training plots
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])

## Evaluation 5 min valuation 5 min Arbitrage



In [ ]:
#loading weights of best previously trained model
#set up model in RTP+DAP tab under training
model_name = net + '_best.hdf5'
dir = folder + region + '_tests\\' + test + '\\' + model_name
model.load_weights(dir)

In [ ]:
# CNNLSTM evaluate function automatically extracts unseen test data from
# full data vector
# need to run this to extract 5 min value prediction for later downsampling
# and hour prediction
print('NYC PREDICTED:')
v1_NYC, arb1_NY, _ = CNNLSTM_evaluate(model, DAP_NYC, RTP_NYC, 
                                    startTest, stopTest, lastDay,
                                    num_DAP, num_RTP, Pr=Pr)
print('\nLONGIL PREDICTED:')
v1_LONGIL, arb1_LONGIL, _  = CNNLSTM_evaluate(model, DAP_LONGIL, RTP_LONGIL, 
                                          startTest, stopTest, lastDay, 
                                          num_DAP, num_RTP, Pr=Pr)
print('\nNORTH PREDICTED:')
v1_NORTH, arb1_NORTH, _ = CNNLSTM_evaluate(model, DAP_NORTH, RTP_NORTH, 
                                        startTest, stopTest, lastDay, 
                                        num_DAP, num_RTP, Pr=Pr)
print('\nWEST PREDICTED:')
v1_WEST, arb1_WEST, _ = CNNLSTM_evaluate(model, DAP_WEST, RTP_WEST, 
                                      startTest, stopTest, lastDay, 
                                      num_DAP, num_RTP, Pr=Pr)
#function outputs T_CNN, but to avoid having to run this everytime, T_CNN is
  #also declared in prep work

NYC PREDICTED:
Evaluating using X_test
Profit:  1959
Revenue:  2547
59
Time: 7.73003363609314

LONGIL PREDICTED:
Evaluating using X_test
Profit:  3768
Revenue:  4670
90
Time: 5.875274419784546

NORTH PREDICTED:
Evaluating using X_test
Profit:  2089
Revenue:  2895
81
Time: 5.438001871109009

WEST PREDICTED:
Evaluating using X_test
Profit:  4001
Revenue:  5000
100
Time: 5.904769659042358


In [ ]:
'''
optimal profits to beat, T_CNN is received from CNNLSTM_evaluate function to 
make sure that the optimal and prediction are evaluated for the same time steps
'''
print('NYC OPTIMAL:')

evaluate_using_v(rtp_NYC, vAvg_NYC, 
                 eta, c, T_CNN,
                 Ts, Pr)
print('\nLONGIL OPTIMAL:')
evaluate_using_v(rtp_LONGIL, vAvg_LONGIL, 
                 eta, c, T_CNN,
                 Ts, Pr)
print('\nNORTH OPTIMAL:')
evaluate_using_v(rtp_NORTH, vAvg_NORTH, 
                 eta, c, T_CNN,
                 Ts, Pr)
print('\nWEST OPTIMAL:')
evaluate_using_v(rtp_WEST, vAvg_WEST, 
                 eta, c, T_CNN,
                 Ts, Pr)

NYC OPTIMAL:
2422
3412
99
Time: 5.572420358657837

LONGIL OPTIMAL:
4715
6093
138
Time: 5.540761947631836

NORTH OPTIMAL:
2408
3650
124
Time: 5.604130029678345

WEST OPTIMAL:
4755
6116
136
Time: 5.561675548553467


4755.395529360187